<a href="https://colab.research.google.com/github/IliaLapushanskyy/Sentiment_headlines_analysis/blob/main/Sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка библиотек

In [71]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import defaultdict
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC

# Загрузка данных и первичный анализ данных

In [72]:
df = pd.read_csv('train.csv', on_bad_lines='warn', sep = ';')

print(df.columns)
print(df.head())

Index(['id', 'text', 'sber', 'vtb', 'gazprom', 'alfabank', 'raiffeisen',
       'rshb', 'company'],
      dtype='object')
      id                                               text  sber  vtb  \
0   8292  #novosti Сбербанк и ВТБ пообещали выполнить об...   1.0  1.0   
1  10897  Корея может стать следующей Японией для инвест...   NaN  NaN   
2   3964  #Новые санкции затрагивают и Газпромбанк  #sledui   NaN  NaN   
3  11829  VEON заявила об обесценивании бизнеса в России...   NaN  NaN   
4  18814                          банк втб ипотечный кредит   NaN  0.0   

   gazprom  alfabank  raiffeisen  rshb  company  
0      NaN       NaN         NaN   NaN      NaN  
1      NaN       NaN         NaN   NaN      NaN  
2     -1.0       NaN         NaN   NaN      NaN  
3      NaN       NaN         NaN   NaN     -1.0  
4      NaN       NaN         NaN   NaN      NaN  


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7161 entries, 0 to 7160
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          7161 non-null   int64  
 1   text        7161 non-null   object 
 2   sber        3344 non-null   float64
 3   vtb         1478 non-null   float64
 4   gazprom     404 non-null    float64
 5   alfabank    654 non-null    float64
 6   raiffeisen  273 non-null    float64
 7   rshb        468 non-null    float64
 8   company     743 non-null    float64
dtypes: float64(7), int64(1), object(1)
memory usage: 503.6+ KB


In [74]:
print("\nПропуски:")
print(df.isna().sum())


Пропуски:
id               0
text             0
sber          3817
vtb           5683
gazprom       6757
alfabank      6507
raiffeisen    6888
rshb          6693
company       6418
dtype: int64


Также важно понимать, сколько и каких значений мы ожидаем увидеть для разных компаний

In [75]:
sentiment_cols = ['sber', 'vtb', 'gazprom', 'alfabank', 'raiffeisen', 'rshb', 'company']

for col in sentiment_cols:
    print(f"\n{col} sentiment distribution:")
    print(df[col].value_counts(dropna=False))


sber sentiment distribution:
sber
 NaN    3817
 0.0    2069
 1.0     648
-1.0     627
Name: count, dtype: int64

vtb sentiment distribution:
vtb
 NaN    5683
 0.0    1107
-1.0     284
 1.0      87
Name: count, dtype: int64

gazprom sentiment distribution:
gazprom
 NaN    6757
 0.0     232
-1.0     130
 1.0      42
Name: count, dtype: int64

alfabank sentiment distribution:
alfabank
 NaN    6507
 0.0     513
-1.0     100
 1.0      41
Name: count, dtype: int64

raiffeisen sentiment distribution:
raiffeisen
 NaN    6888
 0.0     240
-1.0      18
 1.0      15
Name: count, dtype: int64

rshb sentiment distribution:
rshb
 NaN    6693
 0.0     304
-1.0     126
 1.0      38
Name: count, dtype: int64

company sentiment distribution:
company
 NaN    6418
 1.0     273
 0.0     258
-1.0     212
Name: count, dtype: int64


# Предобработка текста

Подключим NLTK (Natural Language Toolkit) для того чтобы лучше обрабатывать русский язык

In [76]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

stemmer = SnowballStemmer("russian")
stop_words = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [77]:
df['clean_text'] = df['text'].astype(str)


# Приводим к строковому типу
df['clean_text'] = df['text'].astype(str)

# Приведение к нижнему регистру
df['clean_text'] = df['clean_text'].str.lower()

# Удаление ссылок
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x))

# Удаление чисел
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'\d+', '', x))

# Удаление пунктуации
df['clean_text'] = df['clean_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Удаление лишних пробелов
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Токенизация
df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join([stemmer.stem(token) for token in word_tokenize(x)
                                                             if token not in stop_words and len(token) > 2]))
df[['text', 'clean_text']].head()

,text,clean_text
0,#novosti Сбербанк и ВТБ пообещали выполнить об...,novosti сбербанк втб пообеща выполн обязательс...
1,Корея может стать следующей Японией для инвест...,коре стат след япон инвестор
2,#Новые санкции затрагивают и Газпромбанк #sledui,нов санкц затрагива газпромбанк sledui
3,VEON заявила об обесценивании бизнеса в России...,veon заяв обесцениван бизнес росс млн квартал
4,банк втб ипотечный кредит,банк втб ипотечн кред


# Векторизация

In [97]:
stop_words_list = list(stop_words)

vectorizer = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 3),
    min_df=3,
    max_df=0.9,
    stop_words=stop_words_list
)
X_tfidf = vectorizer.fit_transform(df['clean_text'])
print("Размер матрицы TF-IDF:", X_tfidf.shape)

Размер матрицы TF-IDF: (7161, 5841)


# Обучение модели

In [82]:
targets = ['sber', 'vtb', 'gazprom', 'alfabank', 'raiffeisen', 'rshb', 'company']

models = {}
reports = {}

for col in targets:
    print(f'\n          Обучение модели для: {col}    ')

    # Только строки с валидными метками
    df_col = df[df[col].notnull()].copy()
    y = df_col[col].astype(int)
    X = vectorizer.transform(df_col['clean_text'])

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Балансировка классов
    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X_train, y_train)

    model = LogisticRegression(max_iter=1000, class_weight='balanced')
    model.fit(X_res, y_res)

    y_pred = model.predict(X_val)
    report = classification_report(y_val, y_pred, digits=3, output_dict=True)
    print(classification_report(y_val, y_pred, digits=3))

    models[col] = model
    reports[col] = report


          Обучение модели для: sber    
              precision    recall  f1-score   support

          -1      0.752     0.728     0.740       125
           0      0.889     0.855     0.872       414
           1      0.693     0.800     0.743       130

    accuracy                          0.821       669
   macro avg      0.778     0.794     0.785       669
weighted avg      0.826     0.821     0.822       669


          Обучение модели для: vtb    
              precision    recall  f1-score   support

          -1      0.869     0.930     0.898        57
           0      0.968     0.950     0.959       222
           1      0.647     0.647     0.647        17

    accuracy                          0.929       296
   macro avg      0.828     0.842     0.835       296
weighted avg      0.930     0.929     0.929       296


          Обучение модели для: gazprom    
              precision    recall  f1-score   support

          -1      0.958     0.885     0.920        26
    